In [324]:
import numpy as np
import pandas as pd
from time import time
import warnings
warnings.filterwarnings(action="ignore", category=DeprecationWarning)

In [325]:
FIRST_SIZE = 2
LAST_SIZE = 32

COUNT_TRIES = 10

MAX_DEF_COUNT_ITER = 100000
"""Максимальное количество итераций по-умолчанию"""
DEF_EPS = 0.0001
"""Точность по-умолчанию"""

'Точность по-умолчанию'

In [326]:
df = pd.DataFrame(
    columns=[
        "size",
        "true_value_max",
        "true_value_min",
        "value_max",
        "value_max_time",
        "value_min_def",
        "value_min_def_time",
        # "value_min_rel",
        # "value_min_rel_time"
    ],
    index=range(1, (LAST_SIZE - FIRST_SIZE + 1) * COUNT_TRIES + 1),
)
df

,size,true_value_max,true_value_min,value_max,value_max_time,value_min_def,value_min_def_time
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
306,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Подготовка

In [327]:
def calc(A: np.matrix, b: np.matrix) -> np.matrix:
    if b is None:
        print("Вектор собственного значения не был посчитан")
        return None
    return float((b.T @ A @ b) / (b.T @ b))

In [328]:
def output(
    A: np.matrix,
    texts,
    funcs,

    num_iterrations: int = MAX_DEF_COUNT_ITER,
    b_k: np.matrix = None,
    mu: float = None,
) -> tuple[np.matrix, list[float | None]]:

    abs_v = np.abs(np.linalg.eigvals(A))

    abs_v_max = np.max(abs_v)

    abs_v_min = np.min(abs_v)
    # print(

    #     "Собственные значения матрицы A",

    #     f"Максимальное: {abs_v_max}",
    #     f"Минимальное: {abs_v_min}",
    #     sep="\n",
    # )


    # print("========")

    mass_eval = []


    for text, f in zip(texts, funcs):
        start_time = time()
        # print(text)
        if f.__name__ != inverse_power_rel.__name__:
            eval = f(A=A, num_iterations=num_iterrations, b_k=b_k)
            # evec = f(A=A, num_iterations=num_iterrations, b_k=b_k)
            # eval = calc(A, evec)
        else:
            evec, eval = f(A=A, b_k=b_k, mu=mu)
        stop_time = time()
        # print("========")
        mass_eval.append(abs(eval))
        mass_eval.append(stop_time - start_time)


    # print(text_1)

    # first_evec = f_1(A=A, num_iterations=num_iterrations, b_k=b_k)

    # first_eval = calc(A, first_evec)
    # print("========")


    # print(text_2)

    # if f_2.__name__ != inverse_power_rel.__name__:

    #     second_evec = f_2(A=A, num_iterations=num_iterrations, b_k=b_k)

    #     second_eval = calc(A, second_evec)

    # else:

    #     second_evec, second_eval = f_2(A=A, b_k=b_k, mu=mu)
    # print("========")

    # print(


    #     "Собственные векторы:",

    #     f"{text_1}:",
    #     first_evec,


    #     f"{text_2}:",

    #     second_evec,

    #     sep="\n",
    # )
    # print(


    #     "Собственные значения:",

    #     f"{text_1}: {first_eval}",

    #     f"{text_2}: {second_eval}",

    #     sep="\n",
    # )


    return A.shape[0], abs_v_max, abs_v_min, *mass_eval

# Прямой метод

$x^{(k)} = \dfrac{Ax^{(k - 1)}}{\alpha_{k - 1}}$

$\lambda_{1}(A) = \dfrac{\left( Ax^{(k)}, x^{(k)} \right)}{(x^{(k)}, x^{(k)})} = \dfrac{(x^{(k)})^{T}A^{(k)}x^{(k)}}{(x^{(k)})^{T}x^{(k)}}$, где $|\lambda_1(A)|$ - наибольшее по модулю СЗ

In [329]:
# TODO Сделать вычисление СЗ на каждом шаге.

def power_iteration(A: np.matrix, epsilon: float = DEF_EPS, num_iterations = MAX_DEF_COUNT_ITER, b_k: np.matrix = None) -> np.matrix:
    if b_k is None:
        b_k = np.random.rand(A.shape[1], 1)
    # print("Начальное приближение:", b_k, sep="\n")
    
    alpha_old = None

    for _ in range(num_iterations):
        b_k1 = np.dot(A, b_k)

        b_k1_norm = np.linalg.norm(b_k1)

        b_k = b_k1 / b_k1_norm
        
        alpha_new = calc(A, b_k)
        if alpha_old is not None and abs(alpha_new - alpha_old) < epsilon:
            break
        alpha_old = alpha_new
        

    return alpha_new

# Обратный метод

## Стандартная версия

In [330]:
def inverse_power_def(A: np.matrix, epsilon: float = DEF_EPS, num_iterations = MAX_DEF_COUNT_ITER, b_k: np.matrix = None) -> np.matrix:
    try:
        inv_A = np.linalg.inv(A)
    except np.linalg.LinAlgError:
        print("Вырожденная матрица")
        return None
    # print("Обратная матрица:",inv_A, sep="\n")
    return power_iteration(inv_A, epsilon=epsilon, num_iterations=num_iterations, b_k=b_k)

In [331]:
# A: np.matrix = np.matrix("1 3 -2 0;"
#                          "1 1 4 7;"
#                          "4 7 11 23;"
#                          "52 66 2 0")  # -0.65
# calc(A, inverse_power_def(A))

## Модификация Релэя

In [332]:
import random


def inverse_power_rel(A, epsilon: float = DEF_EPS, num_iterations = MAX_DEF_COUNT_ITER, mu = None, b_k = None):
  if b_k is None:
      b_k = np.random.rand(A.shape[1], 1) # Попробовать близкий к настоящему собственный вектор
  if mu is None:
    mu = random.randint(1, 19) / 10 # Случайное начальное приближение
  print("Начальное приближение:", "СЗ:", mu, "СВ:", b_k, sep="\n")
  
  b_k = b_k / np.linalg.norm(b_k)
  y = np.linalg.solve(A - mu * np.eye(A.shape[0]), b_k)
  # print(y)
  # print(x)
  lambda_ = float(np.dot(y.T, b_k))
  # print(lambda_, lambda_ * x)
  mu = mu + 1 / lambda_
  err = np.linalg.norm(y - lambda_ * b_k) / np.linalg.norm(y)
  
  index = 0
  while err > epsilon or index < num_iterations:
    b_k = y / np.linalg.norm(y)
    y = np.linalg.solve(A - mu * np.eye(A.shape[0]), b_k)
    lambda_ = float(np.dot(y.T, b_k))
    mu = mu + 1 / lambda_
    err = np.linalg.norm(y - lambda_ * b_k) / np.linalg.norm(y)
    index += 1

  return b_k, float(mu)


In [333]:
A: np.matrix = np.matrix("1 -2 -1;"
                         "-1 1 1;"
                         "1 0 -1")  # 0, 2, -1
# Сильно зависит от начального приближения
# Добавить анализ через Pandas
# A, calc(A, power_iteration(A, 1000)), inverse_power_rel(A, 0.0001)[1]
# calc(A, inverse_power_rel(A)[0])

In [334]:
def generate_non_singular(n):
    while True:
        # Генерируем случайную матрицу n x n
        matrix = np.random.rand(n, n)  # Вы можете использовать np.random.randint для целых чисел.
        if np.linalg.det(matrix) != 0:  # Проверяем, что определитель не равен нулю
            return matrix

In [335]:
import numpy as np
# texts = ["Прямые итерации", "Обратные итерации (обычные)", "Обратные итерации (Релэя)"]
# funcs = [power_iteration, inverse_power_def, inverse_power_rel]
texts = ["Прямые итерации", "Обратные итерации (обычные)"]
funcs = [power_iteration, inverse_power_def]
mass = []

for i in range(FIRST_SIZE, LAST_SIZE + 1):
    print(f"Current size:{i}")
    for j in range(COUNT_TRIES):
        print(j + 1)
        matr = generate_non_singular(i)
        df.iloc[(i - FIRST_SIZE) * COUNT_TRIES + j] = output(matr, texts, funcs)
    print()
df

Current size:2
1
2
3
4
5
6
7
8
9
10

Current size:3
1
2
3
4
5
6
7
8
9
10

Current size:4
1
2
3
4
5
6
7
8
9
10

Current size:5
1
2
3
4
5
6
7
8
9
10

Current size:6
1
2
3
4
5
6
7
8
9
10

Current size:7
1
2
3
4
5
6
7
8
9
10

Current size:8
1
2
3
4
5
6
7
8
9
10

Current size:9
1
2
3
4
5
6
7
8
9
10

Current size:10
1
2
3
4
5
6
7
8
9
10

Current size:11
1
2
3
4
5
6
7
8
9
10

Current size:12
1
2
3
4
5
6
7
8
9
10

Current size:13
1
2
3
4
5
6
7
8
9
10

Current size:14
1
2
3
4
5
6
7
8
9
10

Current size:15
1
2
3
4
5
6
7
8
9
10

Current size:16
1
2
3
4
5
6
7
8
9
10

Current size:17
1
2
3
4
5
6
7
8
9
10

Current size:18
1
2
3
4
5
6
7
8
9
10

Current size:19
1
2
3
4
5
6
7
8
9
10

Current size:20
1
2
3
4
5
6
7
8
9
10

Current size:21
1
2
3
4
5
6
7
8
9
10

Current size:22
1
2
3
4
5
6
7
8
9
10

Current size:23
1
2
3
4
5
6
7
8
9
10

Current size:24
1
2
3
4
5
6
7
8
9
10

Current size:25
1
2
3
4
5
6
7
8
9
10

Current size:26
1
2
3
4
5
6
7
8
9
10

Current size:27
1
2
3
4
5
6
7
8
9
10

Current size:28
1
2


,size,true_value_max,true_value_min,value_max,value_max_time,value_min_def,value_min_def_time
1,2,0.7928,0.094698,0.792805,0.001019,10.559865,0.001001
2,2,0.734491,0.050972,0.734493,0.0,19.618435,0.0
3,2,1.160059,0.727827,1.160025,0.000532,1.373918,0.001038
4,2,0.452112,0.153439,0.452126,0.0,6.517274,0.0
5,2,0.595017,0.07822,0.595027,0.0,12.784459,0.000987
...,...,...,...,...,...,...,...
306,32,15.78693,0.088918,15.786926,0.0,11.246299,0.0233
307,32,15.92023,0.166745,15.920229,0.001067,5.316167,2.961745
308,32,16.309292,0.267698,16.309307,0.0,3.735709,0.034909
309,32,15.948705,0.329288,15.948696,0.000992,2.608249,0.046


In [336]:
(LAST_SIZE - FIRST_SIZE - 1) * COUNT_TRIES + COUNT_TRIES

300

## Хз, что это

In [337]:
import numpy as np
def inverse_iteration(A, mu = None, max_iterations=10000, tolerance=1e-6):
    if mu is None:
        mu = random.randint(1, 19) / 10
    n = A.shape[0]
    # Начальное приближение для собственного вектора (можно взять случайный вектор)
    v = np.random.rand(n)
    # Нормализация вектора
    v /= np.linalg.norm(v)
    for iteration in range(max_iterations):
        try:
            # Решение линейной системы (A - mu * I)v = w, где I - единичная матрица,
            w = np.linalg.solve(A - mu * np.eye(n), v)
        except np.linalg.LinAlgError:
            print("Сингулярная матрица. Попробуйте использовать другое значение mu.")
            return None, None
        # Нормализация вектора w
        v_new = w / np.linalg.norm(w)
        # Проверка на сходимость
        if np.linalg.norm(v_new - v) < tolerance:
            break
        v = v_new
    # Вычисление приближенного собственного значения,
    eigenvalue = mu + (v.T @ A @ v) / (v.T @ v),
    return v, float(eigenvalue[0][0])

In [338]:
inverse_iteration(A)

(array([7.07106781e-01, 1.63268092e-17, 7.07106781e-01]), 0.2)